In [2]:
import numpy as np
import pandas as pd
import os
import ast

In [4]:
filenames = os.listdir('test_img')

gt = pd.read_csv(os.path.join('images','train_labels.csv'))
gt_test = pd.read_csv(os.path.join('images','test_labels.csv'))
pred = pd.read_csv('preds.csv')

gt = gt.append(gt_test)

#n = 10

blocks_pred= []
blocks_gt = []
df= []

for i in range(len(filenames)):
    
    # read gt for each image
    d2 = gt[gt['filename'] == filenames[i]]
    d3 = d2[['xmin','ymin','xmax','ymax']].values
    width = d2['width'].values
    height = d2['height'].values
    
    # read pred for each image
    e2 = pred[pred['filename'] == filenames[i]]
    e3 = e2[['xmin','ymin','xmax','ymax']].values
    classes = e2['class'].values
    
    a1 = d2['filename'].values
    a2 = e2['filename'].values
    
    n = len(d3)
    blocks_gt.append(n)
    k = len(e3)
    blocks_pred.append(k)
    
    #IOU calculation
    for j in range(len(d3)):
        
        [xgmin, ygmin, xgmax, ygmax] = d3[j]
        [XGMIN,YGMIN,XGMAX,YGMAX] = [xgmin,ygmin,xgmax,ygmax]
        
        for k in range(len(e3)):

            [xmin, ymin, xmax, ymax] = e3[k]

            [XMIN,YMIN,XMAX,YMAX] = [xmin,ymin,xmax,ymax]

            xmin = min(xmin,xmax)
            xmax = max(xmin,xmax)
            ymin = min(ymin,ymax)
            ymax = max(ymin,ymax)

            Ag = (xgmax-xgmin) * (ygmax - ygmin)
            Ap = (xmax-xmin) * (ymax - ymin)

            # intersection between predicted and ground truth

            x1 = max(xmin,xgmin)
            x2 = min(xmax,xgmax)
            y1 = max(ymin,ygmin)
            y2 = min(ymax, ygmax)

            if (x2 > x1) and (y2 > y1):

                I = (x2 - x1) * (y2 - y1)
            else:
                I = 0

            U = Ap + Ag - I

            iou = abs(I) / abs(U)
            
            df.append((a2[k], width[j],height[j],classes[k], XMIN,YMIN, XMAX,YMAX,iou))
    
iou_df = pd.DataFrame(df, columns = ('filename','width','height','class','xmin','ymin','xmax','ymax','IOU'))

#drop maize indices as we do not need them for canopy cover calculation
indices = iou_df[iou_df['class'] == 'weed'].index
iou_df = iou_df.drop(indices)

# consider only those bounding box predictions whose IOU > 0.6
iou_df = iou_df[iou_df['IOU'] > 0.6]

iou_df.to_csv('iou_all_images.csv',index=False)
    
print('GT-Boxes:',blocks_gt)
print('PRED-Boxes:',blocks_pred)

GT-Boxes: [13, 39, 52, 33, 37, 30, 43, 11, 54, 39, 27, 26, 34, 24, 32, 21, 32, 23, 28, 22, 48, 19, 28, 28, 14, 31, 27, 15, 12, 21, 17, 19, 25, 38, 22, 50, 14, 15, 48, 23, 45, 20, 39, 20, 32, 20]
PRED-Boxes: [12, 25, 20, 23, 22, 19, 18, 10, 17, 20, 24, 20, 25, 24, 22, 19, 27, 17, 23, 22, 23, 21, 21, 20, 16, 20, 20, 14, 11, 19, 14, 20, 23, 17, 20, 27, 16, 13, 31, 19, 36, 19, 25, 17, 33, 20]
